## Set-up

In [ ]:
import etiq
import numpy as np

In [ ]:
etiq.login("https://dashboard.etiq.ai/", "<token>")

In [ ]:
# Can get/create a single named project
project = etiq.projects.open(name="RCA_Drift_Scans")

## Create the test datasets based on the Adult Income Dataset

To illustrate some of the library's features, we build a model that predicts whether an applicant makes over or under 50K using the Adult dataset from https://archive.ics.uci.edu/ml/datasets/adult.

In [ ]:
# Loading a dataset. We're using the adult dataset
data = etiq.utils.load_sample("adultdata")
data.head()
data = data.replace('?', np.nan)
data.dropna(inplace=True)

In [ ]:

from etiq.transforms import LabelEncoder
import pandas as pd
import numpy as np 

# use a LabelEncoder to transform categorical variables
cont_vars = ['age', 'educational-num', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_vars = list(set(data.columns.values) - set(cont_vars))

label_encoders = {}
data_encoded = pd.DataFrame()
for i in cat_vars:
    label = LabelEncoder()
    data_encoded[i] = label.fit_transform(data[i])
    label_encoders[i] = label

data_encoded.set_index(data.index, inplace=True)
data_encoded = pd.concat([data.loc[:, cont_vars], data_encoded], axis=1).copy()


# Notebook Summary


### Quickstart

  1. Import etiq library - for install please check our docs (https://docs.etiq.ai/)

  2. Login to the dashboard - this way you can send the results to your dashboard instance (Etiq AWS instance if you use the SaaS version). To deploy on your own cloud instance, get in touch (info@etiq.ai)

  3. Create or open a project

### Feature drift (RCA)


  4. Load Adult dataset

  5. Create drifted dataset based on a defined segment of the Adult data - for example purposes

  6. Load your config file and create your snapshot

  7. Scan for RCA feature drift


### Target drift (RCA)

  8. Create drifted datasets based on a defined segment of the Adult datatset - for example purposes

  9. Load your config file and create your snapshot

  10. Scan for feature, concept and target drift

## What is drift?

Drift can impact your model in production and make it perform worse than you initially expected.

There are a few different kinds of drift:

1. Feature drift

Feature drift takes place when the distributions of the input features changes. For instance, perhaps you built your model on a sample dataset from the winter period and it's now summer, and your model predicting what kind of dessert people are more likely to buy is not longer as accurate.


2. Target drift

Similarly to feature drift, target drif is about distribution of the predicted feature changing from one time period to the next.

## What is RCA Drift?

Imagine if your drift tests have picked up issues, this test finds out exactly which segment has the issue, which should help you fix it sooner. In addition if only a part of the data drifted your overall tests might not pick up on it, but this test would. The scan it will auto-discover problematic segments on its own without the need for the user to specify segments to test.

Currently we only provide RCA drift for feature and target drift.

This test pipeline is experimental.




In [ ]:
# Create the "drifted" dataset
data_encoded_comparison = data_encoded.copy()
# Expand hours worked by the self employed by 25% in the comparison dataset
# This is small enough that this drift will not show up for the overall dataset for 'hours-per-week'
self_emp_mask = data_encoded.eval(f'workclass in {list(label_encoders["workclass"].transform(["Self-emp-inc"]))}')
data_encoded_comparison.loc[self_emp_mask, 'hours-per-week'] = 1.25 * data_encoded[self_emp_mask]['hours-per-week']

In [ ]:
# Construct etiq style datasets
base_dataset = etiq.SimpleDatasetBuilder.from_dataframe(data_encoded, target_feature='income').build()
comparison_dataset = etiq.SimpleDatasetBuilder.from_dataframe(data_encoded_comparison, target_feature='income').build()

## Calculate RCA Feature Drift

1. (Optional) Define custom drift measures (See the normal drift measure scan for an example)

2. Loading the config 

3. Log the datasets & create the snapshot - no model needed for this scan

4. Run the RCA feature drift scan

This can happen at any point in the pipeline and through a variety of ways

In [ ]:
with etiq.etiq_config("./drift-config-rca.json"):
    snapshot = project.snapshots.create(name="Test Drift RCA Snapshot", 
                                        dataset=base_dataset,
                                        comparison_dataset=comparison_dataset,
                                        model=None)
    # Scan for feature drift
    (segments, issues, issue_summary) = snapshot.scan_drift_metrics_rca()
    # Scan for target drift
    (segments_t, issues_t, issue_summary_t) = snapshot.scan_target_drift_metrics_rca()

In [ ]:
issues

In [ ]:
segments.iloc[1]

In [ ]:
print(f'Workclass 3 == "{label_encoders["workclass"].inverse_transform([3])[0]}"')

In [ ]:
issue_summary

In [ ]:
issue_summary_t

## Calculate RCA Target Drift

1. (Optional) Define custom drift measures (See the normal drift measure scan for an example)

2. Loading the config 

3. Log the datasets & create the snapshot - no model needed for this scan

4. Run the RCA target drift scan

This can happen at any point in the pipeline and through a variety of ways

In [ ]:
# Create the "drifted" dataset
data_encoded_target_drifted = data_encoded.copy()
# For all pacific islanders slip income
pacific_islander_mask = data_encoded.eval(f'race in {list(label_encoders["race"].transform(["Asian-Pac-Islander"]))}')
data_encoded_target_drifted.loc[pacific_islander_mask, 'income'] = np.logical_not(data_encoded[pacific_islander_mask]['income']).astype(int)

In [ ]:
comparison_dataset2 = etiq.SimpleDatasetBuilder.from_dataframe(data_encoded_target_drifted, target_feature='income').build()
with etiq.etiq_config("./drift-config-rca.json"):
    snapshot2 = project.snapshots.create(name="Test Drift RCA Snapshot", 
                                        dataset=base_dataset,
                                        comparison_dataset=comparison_dataset2,
                                        model=None)
    # Scan for feature drift
    (segments_2, issues_2, issue_summary_2) = snapshot2.scan_drift_metrics_rca()
    # Scan for target drift
    (segments_t2, issues_t2, issue_summary_t2) = snapshot2.scan_target_drift_metrics_rca()

In [ ]:
issue_summary_t2